In [38]:
# importar bibliotecas
import tweepy
import re
import datetime
import pandas as pd
from datetime import datetime

In [2]:
# Consumer keys e access tokens, para autenticação com OAuth
consumer_key = 'XXXXXXXXX'
consumer_secret = 'XXXXXXXXX'
access_token = 'XXXXXXXXX'
access_token_secret = 'XXXXXXXXX'

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [4]:
# twittes = api.home_timeline()
# for tweet in twittes:
#     print(tweet.text)

In [5]:
#!pip install textblob

In [6]:
from tweepy import Stream
from tweepy import StreamListener
import json
from  textblob import TextBlob
import re
import csv

In [7]:
distribuidoras = {'DistribuidoraTwitterAmazonasEnergia': '@Amazonasenergia', 'CEA': '@CEA_AP', 'CEB': '@CEBDistribuicao', 
         'CEEE': '@CEEE_IMPRENSA', 'Celesc': '@Celescoficial', 'Celpe': '@OficialCelpe', 'Cemig': '@cemig_energia',
         'Cocel': '@cocel_pr', 'Coelba': '@OficialCoelba', 'Copel': '@copelenergiaPR', 'Cosern': '@OficialCosern',
         'CPFL (Grupo)': '@cpflenergia', 'EDP': '@edpbr', 'Elektro': '@OficialElektro', 'Enel (Grupo)': '@EnelClientesBR',
         'Energisa (Grupo)': '@energisa', 'Equatorial Alagoas': '@equatorialAL', 'Equatorial Maranhão': '@equatorial_ma', 
         'Equatorial Pará': '@equatorial_pa', 'Equatorial Piauí': '@equatorial_Pi', 'Light': '@lightclientes', 
         'RGE': '@rge_cpflenergia', 'Roraima Energia': '@Roraima_Energia'}

In [27]:
def gatilho(dist, qtde):
    print('Gatilho ativado com', qtde, 'para a distribuidora ', dist)

In [61]:
Amazonasenergia = 0
CEA_AP = 0
CEBDistribuicao = 0
CEEE_IMPRENSA = 0
Celescoficial = 0
OficialCelpe = 0
cemig_energia = 0
cocel_pr = 0
OficialCoelba = 0
copelenergiaPR = 0
OficialCosern = 0
cpflenergia = 0
edpbr = 0
OficialElektro = 0
EnelClientesBR = 0
energisa = 0
equatorialAL = 0
equatorial_ma = 0
equatorial_pa = 0
equatorial_Pi = 0
lightclientes = 0
rge_cpflenergia = 0
Roraima_Energia = 0

hora = datetime.now().hour

limite = 200

header_name = ['Light','Grupo Enel']

with open('aneel_distribuidoras.csv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames=header_name)
    writer.writeheader()

class Listener(StreamListener):
    
    def on_data(self, data):
        raw_twitts = json.loads(data)
        try:
            tweets = raw_twitts['text'].lower()

            tweets = ' '.join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweets).split())
            tweets = ' '.join(re.sub('RT',' ', tweets).split())

            blob = TextBlob(tweets.strip())

            global lightclientes
            global EnelClientesBR
            global hora
            global limite

            light_conta = 0
            enel_conta = 0
            now = datetime.now().hour
            for sent in blob.sentences:
                if "lightclientes" in sent:
                    enel_conta += 1
                if "enelclientesbr" in sent:
                    light_conta += 1

            lightclientes = lightclientes + light_conta
            EnelClientesBR = EnelClientesBR + enel_conta
            agora = datetime.now().hour
            if (agora > hora or agora == 0):
                if lightclientes >= limite:
                    gatilho('Light',lightclientes)
                    lightclientes = 0
                if EnelClientesBR >= limite:
                    gatilho('Light',EnelClientesBR)
                    EnelClientesBR = 0                
                hora = agora
                lightclientes = 0
                EnelClientesBR = 0

            with open('aneel_distribuidoras.csv', 'a') as file:
                writer = csv.DictWriter(file, fieldnames=header_name)
                info = {
                    'Light': lightclientes,
                    'Grupo Enel': EnelClientesBR
                }
                writer.writerow(info)
            print(tweets)
        except:
            print('Found an Error')
        
    def on_error(self, status):
        print(status)

In [ ]:
twitter_stream = Stream(auth, Listener())
twitter_stream.filter(track = ['@lightclientes', '@EnelClientesBR'])